In [11]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Setup

Authentification, initialize and load libraries

In [1]:
import ee
import requests
import json
import folium

ee_project = 'ee-zoometh'

ee.Authenticate()
ee.Initialize(project = ee_project)

# Satellite imagery and Heritage Place geometry

In [10]:
# URL to the raw GeoJSON file on GitHub
url = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/projects/caravanserail/caravanserail.geojson"
response = requests.get(url)
geojson = json.loads(response.text)
# coordinates
polygon_coords = geojson['features'][0]['geometry']['coordinates']
polygon = ee.Geometry.Polygon(polygon_coords)

# invert Lat Lon and flat list
polygon_coords_i = [[lat, lon] for lon, lat in polygon_coords[0]]
polygon = ee.Geometry.Polygon(polygon_coords)
# polygon = ee.Geometry.Polygon(polygon_coords_i)

# Select a Landsat 8 image.
landsat_image = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(polygon) \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B4', 'B3', 'B2')
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 30000,
    'dimensions': 512,
    'bands': ['B4', 'B3', 'B2'],
}
# Get the URL for the thumbnail of the image.
thumbnail_url = landsat_image.getThumbURL(vis_params)

# Create a folium map object.
map_center = polygon.centroid().coordinates().getInfo()[::-1]
m = folium.Map(location=map_center, zoom_start=17, height='50%')

folium.raster_layers.TileLayer(
    tiles='https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    max_zoom=20,
    subdomains=['mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(m)

# Add the Landsat image to the map object.
m.add_child(folium.raster_layers.ImageOverlay(name='Landsat 8 Image', image=thumbnail_url, bounds=[[29.87959, 59.974786], [29.880725, 59.9761]], opacity=1))
# Add the polygon to the map object.
folium.vector_layers.Polygon(locations=polygon_coords_i, color='blue', fill_color='blue').add_to(m)
# Add layer control
m.add_child(folium.LayerControl())
m